In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import regexp_tokenize
from nltk.stem.porter import PorterStemmer
import time
from scipy.sparse import csr_matrix

In [2]:
start_time = time.time()
train = pd.read_csv("input/train.tsv", sep='\t')
train = train[:10000]
test = pd.read_csv("input/test.tsv", sep='\t')
test = test[:10000]
print('[{}] Finished to load train/test'.format(time.time() - start_time))

[9.972500085830688] Finished to load train/test


In [3]:
def clean_missing(df):
    df["category_name"] = df["category_name"].replace(np.NaN, "")
    df["brand_name"] = df["brand_name"].replace(np.NaN, "")
    df["item_description"] = df["item_description"].replace(np.NaN, "")
    df["item_description"] = df["item_description"].replace(np.NaN, "")
    return df

start_time = time.time()
train = clean_missing(train)
test = clean_missing(test)

print('[{}] Finished to replace NaN'.format(time.time() - start_time))


[0.1830000877380371] Finished to replace NaN


In [4]:
def split_cat(text):
    if text.count('/') > 1:
        return text.split("/")
    else:
        return (["No Label", "No Label", "No Label"])

def transform_category_name(df):
    df['general_cat'], df['subcat_1'], df['subcat_2'] = \
    zip(*df['category_name'].apply(lambda x: split_cat(x)))
    return df

start_time = time.time()
train = transform_category_name(train)
test = transform_category_name(test)

print('[{}] Finished to transform category_name'.format(time.time() - start_time))

[0.12099981307983398] Finished to transform category_name


In [5]:
stop_words = []
def stem_tokenize(text, stop_words=stop_words):
    stemmer = PorterStemmer()
    tokens = regexp_tokenize(text, pattern=r"[A-Za-z]\w+")
    tokens_wo_sw = [x for x in tokens if x not in stop_words and len(x) > 3]
    tokens_stemmed = [stemmer.stem(x) for x in tokens_wo_sw]
    return tokens_stemmed



In [8]:
# category_name :
for xmax in range (50, 100, 5):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=stem_tokenize, decode_error='ignore', strip_accents='unicode', max_df=xmax/100.0, min_df=0)
    X = tfidf_vectorizer.fit_transform(train["category_name"])
    print(X.shape)

(10000, 530)
(10000, 530)
(10000, 530)
(10000, 530)
(10000, 530)
(10000, 530)
(10000, 530)
(10000, 530)
(10000, 530)
(10000, 530)


In [15]:
for xmin in range (1, 30, 5):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=stem_tokenize, decode_error='ignore', strip_accents='unicode', max_df=1.0, min_df=xmin)
    X = tfidf_vectorizer.fit_transform(train["category_name"])
    print("xmin : " + str(xmin) + ", words : " + str(X.shape[1]))

xmin : 1, words : 530
xmin : 6, words : 286
xmin : 11, words : 236
xmin : 16, words : 203
xmin : 21, words : 172
xmin : 26, words : 160


In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=stem_tokenize, decode_error='ignore', strip_accents='unicode', max_df=1.0, min_df=5)
X = tfidf_vectorizer.fit_transform(train["category_name"])
y = train[""]

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score

#df_sel = df_mod[df_mod['categories'].isin(sel_classes)]
train_data,test_data = train_test_split(df_mod, test_size=0.2)
vn = VectorizeNotes()
X,y = vn.fit(train_data, sel_classes)
clf = SVC(C=1. , kernel='linear')
clf.fit(X,y)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
sims = cosine_similarity(X)
sns.set_context("talk")
sns.heatmap(sims, xticklabels=False, yticklabels=False)
plt.show()